In [36]:
%pylab inline
import os
import seaborn as sns
import pandas as pd

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

Populating the interactive namespace from numpy and matplotlib


## import data

In [34]:
TRAIN_FILE = '../../data/train.csv'
TEST_FILE = '../../data/test.csv'
SUB_DIR = '../../submissions/'

In [20]:
train = pd.read_csv(TRAIN_FILE)
test = pd.read_csv(TEST_FILE)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    train['text'], train['author'], test_size=0.33, random_state=42)

In [10]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('chi2', SelectKBest(chi2, k=10000)),
                     ('clf', MultinomialNB()),
])

In [11]:
text_clf.fit(X_train, y_train)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...nction chi2 at 0x1a0e31fb70>)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [12]:
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test) 

0.79340761374187563

In [14]:
y_pred = text_clf.predict_proba(X_test)

In [17]:
from sklearn.metrics import log_loss

In [18]:
log_loss(y_test, y_pred)

0.64437815529217424

## Create submission

In [19]:
text_clf.fit(train['text'], train['author'])  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...nction chi2 at 0x1a0e31fb70>)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [21]:
preds = text_clf.predict_proba(test['text'])

In [27]:
sub = pd.DataFrame({'id':test['id']})

In [31]:
sub = pd.DataFrame({a: preds[:,i] for (i,a) in enumerate(train['author'].unique())})

In [32]:
sub['id'] = test['id']

In [40]:
sub.to_csv(os.path.join(SUB_DIR,'submission1.csv'),index=False)